# I. Lý thuyết

## Câu 1: D

## Câu 2: C

## Câu 3: C

## Câu 4: D

## Câu 5: D

1) Mạng tích chập CNN sẽ được thực hiện như thế nào?

A. Phân chia feature map thành các ma trận theo chiều channel và thực hiện tích chập giữa mỗi ma trận kernel với từng chiều.

B. Thực hiện phép nhân ma trận kernel với chuyển vị của ma trận local receptive field để thu được giá trị feature map output.

C. Tính trung bình các phân tử trên một ma trận local receptive field.

D. Di chuyển vùng local receptive field theo chiều từ trái sang phải và từ trên xuống dưới và nhân tích chập với kernel để thu được ma trận feature map.

2) Thực hiện phép nhân tích chập giữa ma trận đầu vào kích thước width và height lần lượt là $(W, H)$ với bộ lọc kích thước $(F, F)$ và bước nhảy $S$ thì thu được ma trận đầu ra với kích thước width và height $(W', H')$ là bao nhiêu? Hãy lựa chọn công thức tổng quát nhất.

A. $W' =⌊ \frac{W-F}{S} ⌋ + 1;~ H' =⌊ \frac{H-F}{S} ⌋ + 1$

B. $W' =\lceil \frac{W-F}{S} \rceil + 1;~ H' = \lceil \frac{H-F}{S} \rceil + 1$ 

C. $W' =\lfloor \frac{W-F+2P}{S} \rfloor + 1;~ H' = \lfloor \frac{H-F+2P}{S} \rfloor + 1$ với $P$ là số lượng các véc tơ 0 padding ở bên ngoài đều hai phía.

D. $W' =\lceil \frac{W-F+P}{S} \rceil + 1;~ H' = \lceil \frac{H-F+P}{S} \rceil + 1$ với $P$ là số lượng các véc tơ 0 padding ở bên ngoài đều hai phía.

3) Mạng CNN sẽ học được những gì qua các layers?

A. Tại những layers đầu mỗi channels sẽ giúp phân biệt một loại đặc trưng, của vật thể. Những đặc trưng này sau đó được trải phẳng và đưa vào mạng MLP để phân loại vật thể.

B. Những layers cuối cùng sẽ zoom vào từng chi tiết của vật thể để phân loại vật thể.

C. Tại những layers đầu mạng sẽ học được các đặc trưng chung như các edge dọc, ngang, chéo,.... Các đặc trưng chi tiết và tổng quát giúp nhận diện vật thể được học tại những layers cuối cùng.

D. Các layers đầu tiên sẽ tập trung vào các chi tiết bộ phận của vật thể.

4) Kiến trúc chung của một mạng CNN là gì?

A. _[Conv -> BatchNorm -> Activation -> Maxpooling] x n_

B. _[BatchNorm -> Conv -> Activation -> Maxpooling] x n_ 

C. _[Conv -> BatchNorm -> Maxpooling -> Activation -> Maxpooling] x n_ 

D. _[Conv -> BatchNorm -> Maxpooling -> Activation -> Maxpooling] x n -> Flatten -> [FullyConnected -> Activation] x m -> Softmax_ 

5) Thông thường đặc điểm kích thước feature map của mạng neural sẽ như thế nào ?

A. Kích thước feature map tăng dần gấp đôi qua thời gian.

B. Kích thước feature map sẽ duy trì không đổi qua thời gian.

C. Kích thước mạng sẽ giảm dần gấp đôi sau mỗi một lần downsampling và sau đó tăng dần gấp đôi sau mỗi lần upsampling để khôi phục về kích thước ảnh input.

D. Kích thước mạng thường giảm gấp đôi sau một lần downsampling và số lượng các filters tăng dần.

# II. Thực hành

6) Thực hiện xây dựng và huấn luyện một mạng CNN ngẫu nhiên trên bộ dữ liệu [Dog and Cat](https://www.kaggle.com/c/dog-vs-cat-classification/data). Lưu ý cần thực hiện [chuẩn hóa dữ liệu](https://github.com/pytorch/examples/issues/112) đối với bộ dữ liệu ImageNet trước khi huấn luyện.

7) Grid Search kiến trúc CNN dựa trên việc tổng quát hóa một thiết kế câu 6.

8) Huấn luyện lại mô hình bằng kiến trúc ResNet và MobileNet. Đánh giá chi phí tính toán và submit kết quả độ chính xác trên các ảnh thuộc folder test.

9) Thử nghiệm các phương pháp Augmentation như Flip, Rotation, Random Crop, Bright Contrast để cải thiện kết quả mô hình.

10) Lập bảng kết quả các thử nghiệm đã sử dụng.

In [ ]:
from google.colab import drive
import os
drive.mount('/content/gdrive')
os.chdir('gdrive/MyDrive/DeepLearning Course 3')

Mounted at /content/gdrive


## Câu 6

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn.functional as F

In [ ]:
def load_dataset(data_path):
    transformation = transforms.Compose([
        transforms.Resize((128, 128)),                                
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    
    full_dataset = torchvision.datasets.ImageFolder(data_path, transform=transformation)
    
    train_size = int(0.7 * len(full_dataset))
    test_size = len(full_dataset) - train_size
    
    train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])
    train_loader = torch.utils.data.DataLoader(train_dataset, shuffle=True, batch_size=128, num_workers=1)
    test_loader = torch.utils.data.DataLoader(test_dataset, shuffle=False, batch_size=128, num_workers=1)
    
    return train_loader, test_loader

data_path = './dataset/dog-vs-cat-classification/train/train'
train_loader, test_loader = load_dataset(data_path)

In [ ]:
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=3, stride=1, padding=1)
        self.maxpool = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(in_channels=12, out_channels=12, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=12, out_channels=24, kernel_size=3, stride=1, padding=1)
        self.dropout = nn.Dropout2d(0.2)
        self.fc = nn.Linear(in_features=32*32*24, out_features=2)
        
    def forward(self, x):
        x = torch.relu(self.maxpool(self.conv1(x)))
        x = torch.relu(self.maxpool(self.conv2(x)))
        x = torch.relu(self.dropout(self.conv3(x)))
        x = x.view(-1, 32*32*24)
        x = self.fc(x)
        return x

In [ ]:
def train(model, device, data_loader, optimizer, loss_criteria):
    model.train()
    train_loss = 0
    
    for batch_idx, (data, target) in enumerate(data_loader):
        data, target = data.to(device), target.to(device)
        output = model(data)
        
        loss = loss_criteria(output, target)
        train_loss += loss.item()
        
        optimizer.zero_grad()

        loss.backward()
        optimizer.step()
    
    avg_loss = train_loss / (batch_idx + 1)
    print('Training set: Average loss: {:.6f}'.format(avg_loss))
    return avg_loss

def test(model, device, data_loader, loss_criteria):
    model.eval()
    test_loss = 0
    correct = 0
    
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(data_loader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += loss_criteria(output, target).item()
            
            _, acc = torch.max(output, axis=1)
            correct += acc.eq(target).sum().item()
            
    avg_loss = test_loss / (batch_idx + 1)
    print('Validation set: Average loss: {:.6f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        avg_loss, correct, len(data_loader.dataset),
        100. * correct / len(data_loader.dataset)))

    return avg_loss

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Training on', device)
model = MyModel().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_criteria = nn.CrossEntropyLoss()

epochs = 5
for epoch in range(epochs):
    print(f'Epoch: {epoch+1}')
    train_loss = train(model, device, train_loader, optimizer, loss_criteria)
    test_loss = test(model, device, test_loader, loss_criteria)


Training on cuda
Epoch: 1
Training set: Average loss: 1.063952
Validation set: Average loss: 0.689644, Accuracy: 4086/7500 (54%)

Epoch: 2
Training set: Average loss: 0.666461
Validation set: Average loss: 0.650250, Accuracy: 4713/7500 (63%)

Epoch: 3
Training set: Average loss: 0.637963
Validation set: Average loss: 0.632951, Accuracy: 4850/7500 (65%)

Epoch: 4
Training set: Average loss: 0.616072
Validation set: Average loss: 0.630774, Accuracy: 4930/7500 (66%)

Epoch: 5
Training set: Average loss: 0.592673
Validation set: Average loss: 0.615037, Accuracy: 5023/7500 (67%)



## Câu 7

In [ ]:
! pip install optuna

     |████████████████████████████████| 308 kB 7.4 MB/s 
     |████████████████████████████████| 210 kB 70.0 MB/s 
     |████████████████████████████████| 81 kB 11.2 MB/s 
     |████████████████████████████████| 75 kB 5.9 MB/s 
     |████████████████████████████████| 113 kB 71.1 MB/s 
     |████████████████████████████████| 150 kB 70.6 MB/s 
     |████████████████████████████████| 49 kB 8.7 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=a4f23c1ee86cf6f81a895b771da4e46110b8d86409f9b0e03bd0b816fb140652
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [ ]:
import optuna
from optuna.trial import TrialState

In [ ]:
device = torch.device('cuda')
def objective(trial):
    model = MyModel().to(device)
    optimizer_name = trial.suggest_categorical('optimizer', ['Adam', 'SGD', 'RMSprop'])
    lr = trial.suggest_float('lr', 1e-5, 1e-2, log=True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr)
    loss_criteria = nn.CrossEntropyLoss()
    
    for epoch in range(5):
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)

            optimizer.zero_grad()
            output = model(data)
            loss = loss_criteria(output, target)
            loss.backward()
            optimizer.step()

        model.eval()
        correct = 0
        with torch.no_grad():
            for batch_idx, (data, target) in enumerate(test_loader):
                data, target = data.to(device), target.to(device)
                output = model(data)
                _, pred = torch.max(output, axis=-1)
                correct += pred.eq(target).sum().item()

        accuracy = correct / len(test_loader.dataset)
        
        trial.report(accuracy, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
    return accuracy

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20, timeout=60*20)

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2022-02-25 11:50:43,348] A new study created in memory with name: no-name-6a91ad30-8273-41f2-ad91-dc1ea65f47b5
[I 2022-02-25 12:01:07,768] Trial 0 finished with value: 0.516 and parameters: {'optimizer': 'SGD', 'lr': 1.0523215010350237e-05}. Best is trial 0 with value: 0.516.
[I 2022-02-25 12:11:35,391] Trial 1 finished with value: 0.7418666666666667 and parameters: {'optimizer': 'Adam', 'lr': 0.00023221319295276067}. Best is trial 1 with value: 0.7418666666666667.


Study statistics: 
  Number of finished trials:  2
  Number of pruned trials:  0
  Number of complete trials:  2
Best trial:
  Value:  0.7418666666666667
  Params: 
    optimizer: Adam
    lr: 0.00023221319295276067


In [ ]:
model = MyModel().to(device)
def train(model, data_loader, optimizer, loss_criteria):
    model.train()
    losses = 0
    for batch, (data, target) in enumerate(data_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()   
        out = model(data)
        loss = loss_criteria(out, target)
        losses += loss.item()
        loss.backward()
        optimizer.step()
    
    avg_loss = losses / (batch + 1)
    print(f"Loss in train set: {avg_loss:.6f}")
    return avg_loss

def evaluation(model, data_loader, optimizer, loss_criteria):
    model.eval()
    losses = 0
    acc = 0
    with torch.no_grad():
        for batch, (data, target) in enumerate(data_loader):
            data, target = data.to(device), target.to(device)
            out = model(data)
            loss = loss_criteria(out, target)
            losses += loss.item()
            _, predict = torch.max(out.data, axis=-1)
            acc += predict.eq(target).sum().item()
    avg_loss = losses / (batch + 1)
    acc = acc * 100 / len(data_loader.dataset)
    print(f"Loss in validation set: {avg_loss:.6f}, Accuracy {acc:.3f}%")
    print('-'*50)
    return avg_loss, acc

OPTIMIZER = getattr(optim, trial.params['optimizer'])(model.parameters(), lr=trial.params['lr'])
LOSS_CRITERIA = nn.CrossEntropyLoss()
MAX_ACC = -float('inf')
ERROR_GOING_UP = 0
EPOCHS = 10

for epoch in range(1, EPOCHS+1):
    print(f'Epoch {epoch}')
    train_loss = train(model, train_loader, OPTIMIZER, LOSS_CRITERIA)
    val_loss, acc = evaluation(model, test_loader, OPTIMIZER, LOSS_CRITERIA)

    if acc > MAX_ACC:
        MAX_ACC = acc
        ERROR_GOING_UP = 0
    else:
        ERROR_GOING_UP += 1
        if ERROR_GOING_UP == 4:
            break
print(f"Best accuracy: {MAX_ACC:.3f}")

Epoch 1
Loss in train set: 0.643635
Loss in validation set: 0.593296, Accuracy 69.413%
--------------------------------------------------
Epoch 2
Loss in train set: 0.590906
Loss in validation set: 0.570504, Accuracy 70.333%
--------------------------------------------------
Epoch 3
Loss in train set: 0.561892
Loss in validation set: 0.549830, Accuracy 72.453%
--------------------------------------------------
Epoch 4
Loss in train set: 0.540264
Loss in validation set: 0.536623, Accuracy 73.507%
--------------------------------------------------
Epoch 5
Loss in train set: 0.520628
Loss in validation set: 0.525562, Accuracy 74.000%
--------------------------------------------------
Epoch 6
Loss in train set: 0.503667
Loss in validation set: 0.509816, Accuracy 74.947%
--------------------------------------------------
Epoch 7
Loss in train set: 0.492557
Loss in validation set: 0.497975, Accuracy 75.920%
--------------------------------------------------
Epoch 8
Loss in train set: 0.48051

## Câu 8

In [ ]:
import tensorflow_hub as hub
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
IMAGE_SHAPE = (224, 224)
BATCH_SIZE = 64

data_path = './dataset/dog-vs-cat-classification/train/train'

data_gen = ImageDataGenerator(rescale=1/255., validation_split=0.2)

print("Training data:")
train_data = data_gen.flow_from_directory(data_path,
                                        target_size=IMAGE_SHAPE,
                                        batch_size=BATCH_SIZE,
                                        class_mode='binary',
                                        shuffle=True,
                                        seed=42,
                                        subset='training')
                                           

print("Testing data")
val_data = data_gen.flow_from_directory(data_path,
                                    target_size=IMAGE_SHAPE,
                                    batch_size=BATCH_SIZE,
                                    class_mode='binary',
                                    subset='validation')

Training data:
Found 20000 images belonging to 2 classes.
Testing data
Found 5000 images belonging to 2 classes.


In [ ]:
# Pretrained model URL
resnet_url = 'https://tfhub.dev/tensorflow/resnet_50/feature_vector/1'
mobilenet_url = 'https://tfhub.dev/google/imagenet/mobilenet_v3_large_100_224/feature_vector/5'

In [ ]:
def create_model(model_url):
    feature_extraction_layer = hub.KerasLayer(model_url,
                                              trainable=False,
                                              input_shape=(IMAGE_SHAPE + (3,)))
    model = tf.keras.models.Sequential([feature_extraction_layer,
                                        tf.keras.layers.Dense(16, activation='relu'),
                                        tf.keras.layers.Dense(2, activation='softmax')])
    
    return model

### Resnet Model

In [ ]:
resnet_model = create_model(resnet_url)
resnet_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 2048)              23561152  
                                                                 
 dense (Dense)               (None, 16)                32784     
                                                                 
 dense_1 (Dense)             (None, 2)                 34        
                                                                 
Total params: 23,593,970
Trainable params: 32,818
Non-trainable params: 23,561,152
_________________________________________________________________


In [ ]:
resnet_model.compile(loss='sparse_categorical_crossentropy',
                     optimizer=tf.keras.optimizers.Adam(),
                     metrics=['acc'])

In [ ]:
resnet_history = resnet_model.fit(train_data,
                                  epochs=3, 
                                  validation_data=val_data)

Epoch 1/3
313/313 [==============================] - 9658s 31s/step - loss: 0.0439 - acc: 0.9846 - val_loss: 0.0456 - val_acc: 0.9882
Epoch 2/3
313/313 [==============================] - 169s 539ms/step - loss: 0.0246 - acc: 0.9916 - val_loss: 0.0370 - val_acc: 0.9884
Epoch 3/3
313/313 [==============================] - 168s 537ms/step - loss: 0.0165 - acc: 0.9942 - val_loss: 0.0357 - val_acc: 0.9902


### MobileNet Model

In [ ]:
mobilenet_model = create_model(mobilenet_url)

In [ ]:
mobilenet_model.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_13 (KerasLayer)  (None, 1280)             4226432   
                                                                 
 dense_20 (Dense)            (None, 16)                20496     
                                                                 
 dense_21 (Dense)            (None, 2)                 34        
                                                                 
Total params: 4,246,962
Trainable params: 20,530
Non-trainable params: 4,226,432
_________________________________________________________________


In [ ]:
mobilenet_model.compile(loss='sparse_categorical_crossentropy',
                     optimizer=tf.keras.optimizers.Adam(),
                     metrics=['accuracy'])

In [ ]:
# Fit the model
mobilenet_history = mobilenet_model.fit(train_data,
                                        epochs=8,
                                        validation_data=val_data)

Epoch 1/8
313/313 [==============================] - 168s 516ms/step - loss: 0.0479 - accuracy: 0.9829 - val_loss: 0.0392 - val_accuracy: 0.9880
Epoch 2/8
313/313 [==============================] - 149s 475ms/step - loss: 0.0197 - accuracy: 0.9930 - val_loss: 0.0359 - val_accuracy: 0.9882
Epoch 3/8
313/313 [==============================] - 146s 466ms/step - loss: 0.0116 - accuracy: 0.9966 - val_loss: 0.0404 - val_accuracy: 0.9868
Epoch 4/8
313/313 [==============================] - 147s 469ms/step - loss: 0.0071 - accuracy: 0.9979 - val_loss: 0.0456 - val_accuracy: 0.9876
Epoch 5/8
313/313 [==============================] - 146s 467ms/step - loss: 0.0041 - accuracy: 0.9994 - val_loss: 0.0470 - val_accuracy: 0.9880
Epoch 6/8
313/313 [==============================] - 146s 466ms/step - loss: 0.0021 - accuracy: 0.9998 - val_loss: 0.0514 - val_accuracy: 0.9872
Epoch 7/8
313/313 [==============================] - 146s 466ms/step - loss: 0.0012 - accuracy: 0.9998 - val_loss: 0.0549 - val_ac

## Câu 9 

In [ ]:
data_gen = ImageDataGenerator(rescale=1/255.,
                              rotation_range=45,
                              width_shift_range=0.2,
                              height_shift_range=0.2,
                              zoom_range=0.2,
                              horizontal_flip=True,
                              shear_range=0.1,
                              validation_split=0.2)

train_data_aug = data_gen.flow_from_directory(data_path,
                                          batch_size=BATCH_SIZE,
                                          target_size=IMAGE_SHAPE,
                                          class_mode='binary',
                                          shuffle=True,
                                          subset='training')

val_data = data_gen.flow_from_directory(data_path,
                                        batch_size=BATCH_SIZE,
                                        target_size=IMAGE_SHAPE,
                                        class_mode='binary',
                                        shuffle=False,
                                        subset='validation')


Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


### MobileNet with Augmentation

In [ ]:
mobile_model_with_aug = create_model(mobilenet_url)

mobile_model_with_aug.compile(loss='sparse_categorical_crossentropy',
                     optimizer=tf.keras.optimizers.Adam(),
                     metrics=['accuracy'])

model_checkpoint = tf.keras.callbacks.ModelCheckpoint('mobilenet_best.hdf5',
                                                      verbose=2,
                                                      save_best_only=True,
                                                      mode='min')

mobile_model_with_aug_history = mobile_model_with_aug.fit(train_data, epochs=5, 
                                                           validation_data=val_data,
                                                          callbacks=[model_checkpoint])

Epoch 1/5
391/391 [==============================] - ETA: 0s - loss: 0.0739 - accuracy: 0.9705
Epoch 1: val_loss improved from inf to 0.04755, saving model to mobilenet_best.hdf5
391/391 [==============================] - 525s 1s/step - loss: 0.0739 - accuracy: 0.9705 - val_loss: 0.0476 - val_accuracy: 0.9832
Epoch 2/5
391/391 [==============================] - ETA: 0s - loss: 0.0483 - accuracy: 0.9814
Epoch 2: val_loss did not improve from 0.04755
391/391 [==============================] - 521s 1s/step - loss: 0.0483 - accuracy: 0.9814 - val_loss: 0.0491 - val_accuracy: 0.9794
Epoch 3/5
391/391 [==============================] - ETA: 0s - loss: 0.0447 - accuracy: 0.9830
Epoch 3: val_loss improved from 0.04755 to 0.04314, saving model to mobilenet_best.hdf5
391/391 [==============================] - 522s 1s/step - loss: 0.0447 - accuracy: 0.9830 - val_loss: 0.0431 - val_accuracy: 0.9826
Epoch 4/5
391/391 [==============================] - ETA: 0s - loss: 0.0422 - accuracy: 0.9832
Epoch

## Câu 10: Đánh giá hiệu quả của các Model

In [ ]:
import pandas as pd
import glob

In [ ]:
path_id = [i for i in glob.glob('./dataset/dog-vs-cat-classification/test/test' + "/*.jpg")]
path_id.sort()

In [ ]:
submission = pd.DataFrame({'id': [os.path.basename(i) for i in path_id],
                           'test_image_path': [i for i in path_id]})
submission

,id,test_image_path
0,000000.jpg,./dataset/dog-vs-cat-classification/test/test/...
1,000001.jpg,./dataset/dog-vs-cat-classification/test/test/...
2,000002.jpg,./dataset/dog-vs-cat-classification/test/test/...
3,000003.jpg,./dataset/dog-vs-cat-classification/test/test/...
4,000004.jpg,./dataset/dog-vs-cat-classification/test/test/...
...,...,...
7995,007995.jpg,./dataset/dog-vs-cat-classification/test/test/...
7996,007996.jpg,./dataset/dog-vs-cat-classification/test/test/...
7997,007997.jpg,./dataset/dog-vs-cat-classification/test/test/...
7998,007998.jpg,./dataset/dog-vs-cat-classification/test/test/...


In [ ]:
test_data_gen = ImageDataGenerator(rescale=1/255.)

submission_data = test_data_gen.flow_from_dataframe(
    submission,
    x_col='test_image_path',
    y_col=None,
    class_mode=None,
    batch_size=BATCH_SIZE,
    target_size=IMAGE_SHAPE,
    shuffle=False
)

Found 8000 validated image filenames.


### MobileNet Prediction

In [ ]:
predictions = mobilenet_model.predict(submission_data)
labels = tf.expand_dims(tf.argmax(predictions, axis=1), 1).numpy()

In [ ]:
submission1 = submission.copy()
submission1['labels'] = labels
new_submission = submission1.drop('test_image_path', axis=1)
new_submission.to_csv('my_submission.csv', index=False)

**Accuracy in Kaggle: 1.0000**

### Resnet Prediction

In [ ]:
predictions = resnet_model.predict(submission_data)
labels = tf.expand_dims(tf.argmax(predictions, axis=1), 1).numpy()

In [ ]:
submission2 = submission.copy()
submission2['labels'] = labels
new_submission = submission2.drop('test_image_path', axis=1)
new_submission.to_csv('my_submission2.csv', index=False)

### MobileNet with Augmentation Prediction

In [ ]:
predictions = mobile_model_with_aug_history.predict(submission_data)
labels = tf.expand_dims(tf.argmax(predictions, axis=1), 1).numpy()

In [ ]:
submission3 = submission.copy()
submission3['labels'] = labels
new_submission = submission3.drop('test_image_path', axis=1)
new_submission.to_csv('my_submission3.csv', index=False)

### Bảng kết quả
+ Do em đang ở viện nên mạng nó không được ổn định lắm, nên khi train xong `ResnetModel` với `MobileNet with Augmentation Prediction` thì bị mất kết nối mạng, không còn đủ thời gian để training lại và cho phần prediction, nên em mới chỉ tổng hợp được một kết quả của `MobileNet Model`: Kết quả submit lên Kaggle đạt độ chính xác là 100.0000